In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
# url_2015 = "http://www.pro-football-reference.com/years/2015/draft.htm"
# html = urlopen(url_2015)

url_1967 = "http://www.pro-football-reference.com/years/1967/draft.htm"
html = urlopen(url_1967)

soup = BeautifulSoup(html, "lxml")

## Scrape the links along with the table data

- Grab each row
- For each row grab the text data in td
    - Store the data via lists
- For each row grab the links
    

In [3]:
# Extract the necessary values for the column headers from the table
column_headers = [th.getText() for th in 
                  soup.findAll('tr', limit=2)[1].findAll('th')]
column_headers.extend(["Player_NFL_Link", "Player_NCAA_Link"])

# the data is found from the 3rd row and on
table_rows = soup.select("#drafts tr")[2:] 

In [4]:
type(table_rows)

list

In [5]:
def extract_player_data(table_rows):
    # create the empty list to store the 
    player_data = []
    for row in table_rows:

        # Some player names end with ' HOF', if they do, strip that part away
        # otherwise get the text data from the table row
        player_list = [td.get_text()[:-4] if td.get_text().endswith(" HOF") 
                       else td.get_text() for td in row.find_all("td")]

        # there are some empty table rows (which are the repeated table headers)
        # we skip over those rows and move on to the next one and continue the
        # for loop
        if not player_list:
            continue

        # Same issue as the player_list text
        # If the text ends with " HOF" strip off the last 4 characters
        # and then set the key as that text, otherwise just set the text as
        # the key
        links_dict = {(link.get_text()[:-4] if link.get_text().endswith(" HOF") 
                       else link.get_text()) : link["href"]
                       for link in row.find_all("a", href=True)}

    #     print(links_dict[player_list[3]])

        # add the link associated to the player's pro-football-reference page, 
        # or en empty string if there is no link
        player_list.append(links_dict.get(player_list[3], ""))

        # add the link for the player's college stats or an empty string
        # if ther is no link
        player_list.append(links_dict.get(player_list[-2], ""))

        # Now append the data to list of data
        player_data.append(player_list)
    return player_data

In [6]:
data = extract_player_data(table_rows)

In [7]:
df = pd.DataFrame(data, columns=column_headers)

In [8]:
df.head()

,Rnd,Pick,Tm,,Pos,Age,To,AP1,PB,St,...,TD,Rec,Yds,TD,Int,Sk,College/Univ,,Player_NFL_Link,Player_NCAA_Link
0,1,1,BAL,Bubba Smith,DE,22,1976,1,2,6,...,,,,,,,Michigan St.,College Stats,/players/S/SmitBu00.htm,http://www.sports-reference.com/cfb/players/bu...
1,1,2,MIN,Clint Jones,RB,22,1973,0,0,2,...,20,38,431,0,,,Michigan St.,College Stats,/players/J/JoneCl00.htm,http://www.sports-reference.com/cfb/players/cl...
2,1,3,SFO,Steve Spurrier,QB,22,1976,0,0,6,...,2,,,,,,Florida,College Stats,/players/S/SpurSt00.htm,http://www.sports-reference.com/cfb/players/st...
3,1,4,MIA,Bob Griese,QB,22,1980,2,8,12,...,7,,,,,,Purdue,College Stats,/players/G/GrieBo00.htm,http://www.sports-reference.com/cfb/players/bo...
4,1,5,HOU,George Webster,LB,21,1976,3,3,6,...,,,,,5,,Michigan St.,College Stats,/players/W/WebsGe00.htm,http://www.sports-reference.com/cfb/players/ge...


In [9]:
df.insert(24, "Tkl", "")

In [10]:
df.head()

,Rnd,Pick,Tm,,Pos,Age,To,AP1,PB,St,...,Rec,Yds,TD,Tkl,Int,Sk,College/Univ,,Player_NFL_Link,Player_NCAA_Link
0,1,1,BAL,Bubba Smith,DE,22,1976,1,2,6,...,,,,,,,Michigan St.,College Stats,/players/S/SmitBu00.htm,http://www.sports-reference.com/cfb/players/bu...
1,1,2,MIN,Clint Jones,RB,22,1973,0,0,2,...,38,431,0,,,,Michigan St.,College Stats,/players/J/JoneCl00.htm,http://www.sports-reference.com/cfb/players/cl...
2,1,3,SFO,Steve Spurrier,QB,22,1976,0,0,6,...,,,,,,,Florida,College Stats,/players/S/SpurSt00.htm,http://www.sports-reference.com/cfb/players/st...
3,1,4,MIA,Bob Griese,QB,22,1980,2,8,12,...,,,,,,,Purdue,College Stats,/players/G/GrieBo00.htm,http://www.sports-reference.com/cfb/players/bo...
4,1,5,HOU,George Webster,LB,21,1976,3,3,6,...,,,,,5,,Michigan St.,College Stats,/players/W/WebsGe00.htm,http://www.sports-reference.com/cfb/players/ge...


# Doing it for all seasons since 1967

In [11]:
# Pass in the draft year to this url template
url_template = "http://www.pro-football-reference.com/years/{year}/draft.htm"

In [12]:
# Create an empty list that will contain all the dataframes
# (one dataframe for each draft)
draft_dfs_list = []

# list to place any errors that may come up while scraping
errors_list = []

In [13]:
# for each year from 1967 to (and including) 2016
for year in range(1967, 2017): 
    
    # Use try/except block to catch and inspect any urls that cause an error
    try:
        # get the draft url
        url = url_template.format(year=year)

        # get the html
        html = urlopen(url)

        # create the BeautifulSoup object
        soup = BeautifulSoup(html, "lxml") 

        # get the column headers
        column_headers = [th.getText() for th in 
                          soup.findAll('tr', limit=2)[1].findAll('th')]
        column_headers.extend(["Player_NFL_Link", "Player_NCAA_Link"])

        # select the data from the table using the '#drafts tr' CSS selector
        table_rows = soup.select("#drafts tr")[2:] 

        # extract the player data from the table rows
        player_data = extract_player_data(table_rows)

        # create the dataframe for the current years draft
        year_df = pd.DataFrame(player_data, columns=column_headers)

        # if it is a draft from before 1994 then add a Tkl column at the 24th column
        if year < 1994:
            year_df.insert(24, "Tkl", "")

        # add the year of the draft to the dataframe
        year_df.insert(0, "Draft_Yr", year)

        # append the current dataframe to the list of dataframes
        draft_dfs_list.append(year_df)
    
    except Exception as e:
        # Store the url and the error it causes in a list
        error =[url, e] 
        # then append it to the list of errors
        errors_list.append(error)


In [14]:
len(errors_list)

0

In [15]:
len(draft_dfs_list)

50

# Now Concatenate the dataframes we scraped

In [16]:
draft_df = pd.concat(draft_dfs_list, ignore_index=True)

In [17]:
draft_df.head()

,Draft_Yr,Rnd,Pick,Tm,,Pos,Age,To,AP1,PB,...,Rec,Yds,TD,Tkl,Int,Sk,College/Univ,,Player_NFL_Link,Player_NCAA_Link
0,1967,1,1,BAL,Bubba Smith,DE,22,1976,1,2,...,,,,,,,Michigan St.,College Stats,/players/S/SmitBu00.htm,http://www.sports-reference.com/cfb/players/bu...
1,1967,1,2,MIN,Clint Jones,RB,22,1973,0,0,...,38,431,0,,,,Michigan St.,College Stats,/players/J/JoneCl00.htm,http://www.sports-reference.com/cfb/players/cl...
2,1967,1,3,SFO,Steve Spurrier,QB,22,1976,0,0,...,,,,,,,Florida,College Stats,/players/S/SpurSt00.htm,http://www.sports-reference.com/cfb/players/st...
3,1967,1,4,MIA,Bob Griese,QB,22,1980,2,8,...,,,,,,,Purdue,College Stats,/players/G/GrieBo00.htm,http://www.sports-reference.com/cfb/players/bo...
4,1967,1,5,HOU,George Webster,LB,21,1976,3,3,...,,,,,5,,Michigan St.,College Stats,/players/W/WebsGe00.htm,http://www.sports-reference.com/cfb/players/ge...


In [18]:
draft_df.tail()

,Draft_Yr,Rnd,Pick,Tm,,Pos,Age,To,AP1,PB,...,Rec,Yds,TD,Tkl,Int,Sk,College/Univ,,Player_NFL_Link,Player_NCAA_Link
15840,2016,7,249,SFO,Prince Charles Iworah,CB,,,0,0,...,,,,,,,West. Kentucky,,/players/I/IworPr00.htm,
15841,2016,7,250,CLE,Scooby Wright III,ILB,22,,0,0,...,,,,,,,Arizona,College Stats,/players/W/WrigSc00.htm,http://www.sports-reference.com/cfb/players/sc...
15842,2016,7,251,PHI,Joe Walker,ILB,,,0,0,...,,,,,,,Oregon,,/players/W/WalkJo01.htm,
15843,2016,7,252,CAR,Beau Sandland,TE,23,,0,0,...,,,,,,,Montana St.,,/players/S/SandBe00.htm,
15844,2016,7,253,TEN,Kalan Reed,CB,22,,0,0,...,,,,,,,Southern Miss,,/players/R/ReedKa00.htm,


# Write the raw/dirty data to a csv file

In [19]:
draft_df.to_csv("data/raw_data/draft_RAW.csv", index=False)